In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
conn = sqlite3.connect('finra.db')
cursor = conn.cursor()

In [ ]:
frame = cursor.execute("SELECT disclosures.Allegations FROM disclosures WHERE disclosures.'Broker Comment' LIKE '%INFORMAL%'").fetchall()

In [ ]:
# !pip3 install requests pypdf2
!pip3 install tqdm

In [2]:
with open('metadata2.rtf','rb') as f:
    txt = f.read()
txt = txt.decode("utf-8").split('\n')

In [ ]:
import os
print(os.listdir('/home/anon/Documents/db'))

In [3]:
# txt[txt.find("Disclosure 1 of "):txt.find("Disclosure 1 of ")+10000]
import sqlite3
conn = sqlite3.connect('/home/anon/Documents/db/finra/firms.db')
cursor = conn.cursor()

# frame

In [113]:
from tqdm import tqdm
cursor.execute('DELETE FROM disclosuretext')
conn.commit()
def searchlist(lst,string):
    return [thing for thing in lst if string in thing]
def processtext(sample):
    finlst,elim = [],[]
    keylst = [':','Does the order constitute','Firm Statement','Regulator Statement'] #Start Text 
    for num in range(len(sample)):
        if any([item for item in keylst if item in sample[num]]):
            ### Find Start(element) text and Stop(idx) locations for each field
            element,idx = sample[num] if ('Sought:' not in sample[num] and 'Granted:' not in sample[num]) or ('Relief Sought:' in sample[num]) else ' '.join(sample[num-1:num+1]),num+1
            if 'Sought:' not in element:
#                 and 'Granted:' not in element
                try:
                    remaining = sample[idx]
                    if 'Does the order constitute a' not in remaining:
                        char = ':' 
                        while char not in remaining:
                            idx+=1
                            if 'Sanction(s)/Relief' not in remaining and 'Sanctions Ordered or Relief' not in remaining:
                                element += ' ' + remaining 
                            remaining = sample[idx]
                except:
                    pass
            finlst.append(element)
    finlst = [item.split(':') for item in finlst]
    for x in range(len(finlst)):
        finlst[x] = [item.strip() for item in finlst[x] if item !='']
        if 'Does the order constitute' in finlst[x][0]:
            if 'No' in finlst[x][0]:
                finlst[x] += ['No']
            elif ' Yes ' in finlst[x][0]:
                finlst[x] += ['Yes']
            finlst[x][0] = finlst[x][0].replace(' Yes ','').replace(' No ','').replace('  ','').replace('afinal','a final')
            finlst[x][0]
        if 'Sanction(s)/Relief' in finlst[x][0] and '  ' in finlst[x][0]:
            finlst[x] = [item for item in finlst[x][0].split('  ') if item!='']
            finlst[x][0] += ' Sought'
            finlst[x][1] = finlst[x][1].replace(' Sought','').strip()
        if 'Sanctions Ordered or Relief' in finlst[x][0] and '  ' in finlst[x][0]:
            finlst[x] = ' '.join(finlst[x]).split('  ')
            finlst[x] = [item for item in finlst[x] if item!='']
            finlst[x][0] += ' Granted'
            finlst[x][1] = ' '.join(finlst[x][1:]).replace(' Granted','').strip()
        if 'Firm Statement' in finlst[x][0] or 'Regulator Statement' in finlst[x][0]:
            finlst[x] = [item for item in finlst[x][0].split('  ') if item!='']
        if len(finlst[x]) > 2:
            finlst[x][1] = ' '.join(finlst[x][1:]).strip()
            del finlst[x][2:]
        if finlst[x][0].isupper():
            try:
                elim.append(x)
                finlst[x-1][1] += ' '.join(finlst[x]).strip()
            except:
                pass
    rem = 0
    for num in elim:
        del finlst[num-rem]
        rem+=1
    return finlst
frame = pd.DataFrame()
locs = sorted(list(set([i for i, n in enumerate(txt) if "Disclosure" in n and " of " in n])))
endlst = []
for loc in tqdm(locs):
    if loc != locs[-1]:
        sample = txt[loc+1:locs[locs.index(loc)+1]]
    else:
        sample = txt[loc+1:]
    sample = [item.strip() for item in sample if 'finra.org' not in item and 'All rights reserved.' not in item and item !='' and 'Report about ' not in item and '2020 FINRA' not in item]
    endlst +=processtext(sample)
numframes = pd.DataFrame(endlst)[0].tolist().count('Reporting Source')
numiter = 0
while numiter < numframes:
    start = [i for i, n in enumerate(pd.DataFrame(endlst)[0].tolist()) if n == 'Reporting Source'][numiter]
    try:
        end = [i for i, n in enumerate(pd.DataFrame(endlst)[0].tolist()) if n == 'Reporting Source'][numiter+1]
    except:
        end = len(endlst)
    intframe = pd.DataFrame(endlst[start:end]).drop_duplicates(subset=0).set_index(0).T
    keep_cols = [item[1] for item in cursor.execute('PRAGMA table_info(disclosuretext);').fetchall()]
    missing=[item for item in intframe.columns if item not in keep_cols]
    if missing!=[]:
        print(missing)
#     if missing == ['Granted']:
# #         print(endlst)
#         print(numiter)
    intframe = intframe.drop(columns=missing).reset_index(drop=True)
    for col in [item for item in ['Date Initiated','Resolution Date','Case Initiated','Disposition Date','Date Court Action Filed'] if item in intframe.columns]:
        try:
            intframe[col] = pd.DataFrame([item.split(' ')[0] if pd.isnull(item) == False else item for item in intframe[col].tolist()])
            intframe[col]=pd.to_datetime(intframe[col],errors='coerce')
        except Exception as e:
            print(e)
    for col in ['Sanctions Ordered','Sum of All Relief Awarded','Sanctions Ordered or Relief Granted']:
        if col in intframe.columns:
            try:
                intframe[col+' Amount'] = pd.DataFrame([float(searchlist(item.split(' '),'$')[0].replace('$','').replace(',','')) if len(searchlist(item.split(' '),'$'))>0 and pd.isnull(item) == False else 0 for item in intframe[col].tolist()])
            except Exception as e:
                print(e)
    for floatcol in [item for item in ['Sum of All Relief Awarded','Sum of All Relief Requested'] if item in intframe.columns]:
        intframe[floatcol]=intframe[floatcol].str.replace('$','').str.replace(',','')
        intframe[floatcol] = pd.to_numeric(intframe[floatcol],errors='ignore')
    intframe.to_sql('disclosuretext',conn,if_exists='append',index=False)
    numiter += 1


100%|██████████| 352/352 [00:00<00:00, 4637.60it/s]


['©2020        Date']
['FINRA. All rights Type']
['©2020        Date']
['©2020 Product     Type(s)']
['Does the order constitute a final order based on violations Allany reserved.98 regulations that prohibit Does the order constitute a final order based on violations of any laws or regulations that prohibit fraudulent, manipulative, or deceptive conduct?']
['©2020 Sanction(s)/Relief Sought']
['FINRA. All rights Type']
['Does the order constitute a final order based on violations of any laws or regulations that prohibit fraudulent, ©2020manipulative, deceptive conduct? Does the order constitute a final order based on violations of any laws or regulations that prohibit fraudulent, manipulative, or deceptive conduct?']
['©2020     Details']
['Does the order constitute a final order based on violations of any laws or regulations ©2020 that FINRA. All prohibit fraudulent, manipulative, or Does the order constitute a final order based on violations of any laws or regulations that prohibit fr

In [96]:
# thing = pd.to_datetime(pd.DataFrame([item.split(' ')[0] for item in intframe['Date Initiated'].tolist()]))
# intframe['Date Initiated'] = thing
# [item.split(' ')[0] for item in intframe['Date Initiated'].tolist()]
# thing
# intframe
[item for item in txt if 'Granted:' in item]

# fines

['Granted:                                 Disgorgement/Restitution',
 'Granted:                                 Disgorgement/Restitution',
 'Granted:                                  Cease and Desist/Injunction',
 'Granted:',
 'Granted:                                  Disgorgement/Restitution',
 'Granted:                                 Disgorgement/Restitution']

In [ ]:
# [[thing for thing in item[0].split('  ') if thing !=''] for item in endlst if 'Principal Sanction(s)' in item[0]]
# [item for item in endlst if 'Statement' in item[0]]
# pd.DataFrame(endlst)[0].tolist().count('Reporting Source')
# endlst[-1]
# frame.drop_duplicates()
locs = [i for i, n in enumerate(txt) if "Disclosure" in n and " of " in n]
len(list(set(locs)))
# locs

In [ ]:
# txt[loc+1:locs[locs.index(loc)+1]]
# # [item for item in txt if "Disclosure " in item and " of " in item]
finlst[-1][0].split('  ')

In [ ]:
# loc
locs[locs.index(loc)+1]
locs

In [7]:
import os
from shutil import copyfile

def statuscheck(dest,idx):
    destfin = dest+'/folder_{}'.format(str(idx)) 
    os.mkdir(destfin)
    return destfin
dest = '/home/anon/Documents/db/finra/ziptxt'
source = '/home/anon/Documents/db/finra/pdf'
filenames = sorted([item for item in os.listdir(source) if '.txt' in item])
idx = 0
destfin = statuscheck(dest,idx)
for file in filenames:
    if len(os.listdir(destfin))==1000:
        idx+=1
        destfin = statuscheck(dest,idx)
    copyfile('{}/{}'.format(source,file),'{}/{}'.format(destfin,file))

In [3]:
os.getcwd()

'/home/anon/Documents/repos/pdftext'

In [ ]:
import io
import requests
from PyPDF2 import PdfFileReader
from subprocess import call

# filename = Path('metadata.pdf')
response = requests.get('https://files.brokercheck.finra.org/firm/firm_361.pdf')
call('qpdf --password=%s --decrypt %s %s' %('', 'metadata.pdf', 'metadata_decr.pdf'), shell=True)
# f = io.BytesIO(response.content)
f = open('metadata_decr.pdf','rb')
reader = PdfFileReader(f)

# reader.getNumPages()
page = reader.getPage(10)
print(page.extractText())

# contents = reader.getPage(0).extractText().split('\n')
# with open('metadata.pdf', 'wb') as f:
#     f.write(response.content)

# filename.write_bytes(response.content)

In [ ]:
filename

In [ ]:
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import LTPage
from io import StringIO

def doc_read(path):
    text_list = []

    document = open(path, 'rb')
    string_writer=StringIO()
#     try:    #Create resource manager
    rsrcmgr = PDFResourceManager()
    # Set parameters for analysis.
    laparams = LAParams()
    device = TextConverter(rsrcmgr, string_writer, codec='utf-8', laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.get_pages(document, maxpages=5):
        interpreter.process_page(page)
    text_list += string_writer.getvalue()
    
#     # Create a PDF page aggregator object.
#     device = PDFPageAggregator(rsrcmgr, laparams=laparams)
#     interpreter = PDFPageInterpreter(rsrcmgr, device)
# #          print(PDFPage.get_pages(document)[0])
#     for page in PDFPage.get_pages(document):
#         interpreter.process_page(page)
#         # receive the LTPage object for the page.
#         layout = device.get_result()
#         print(layout)
#         for element in layout:
#             print(element.get_text())
#             try:
#                 text_list.append(element.get_text().replace('\n',''))
#             except:
#                 pass

#     document.close()
#     except Exception as e:
#         document.close()
#         print(e)
    return text_list
doc_read('metadata_decr.pdf')

In [ ]:
from pdfminer.converter import TextConverter
from io import StringIO
path = 'metadata.pdf'
rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
fp = open(path, 'rb')
interpreter = PDFPageInterpreter(rsrcmgr, device)
password = ""
maxpages = 0
caching = True
pagenos=set()
for page in PDFPage.get_pages(fp, pagenos, password=password,maxpages=maxpages,caching=caching, check_extractable=True):
    interpreter.process_page(page)